# Neural Style Transfer using GANs

### Installing and Importing required libraries

In [22]:
#Importing required libraries
import keras
import numpy as np
from keras import Model
import tensorflow as tf
from datetime import datetime
from keras.optimizers import SGD
from keras.utils import plot_model
from keras.preprocessing import image
from tensorflow.keras.applications import vgg19

## 1. Utilizing VGG19 to create style transfer algorithm

### Style Loss Function

In [23]:
#Creating a Gram Matrix
def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

#Creating the loss function by using the Gram Matrix between the result and input image
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

### Content Loss Function

In [24]:
#Creating the loss function
def content_loss(base, combination):
    return tf.reduce_sum(tf.square(combination - base))

### Neural Style Transfer

Loading the pretrained VGG19 model

In [ ]:
#Loading the VGG19 model that has been trained on Imagenet
model = vgg19.VGG19(weights = "imagenet", include_top = False)

#Checking model layers and # of parameters
model.summary()

Calculating the Loss Function

In [26]:
#Extracting values of specific layers
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = Model(inputs = model.inputs, outputs = outputs_dict)

#Defining the style layers to extract
style_layers = [ "block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1", "block5_conv1",]

#Defining the content layer to extract
content_layer = "block5_conv2"

#Assigning weights for content and style loss functions
content_weight = 2.5e-7
style_weight = 1e-6

#Creating a function to calculate the total loss
def loss_function(combination_image, base_image, style_reference_image):

    #Combining all the images in the same tensor
    input_tensor = tf.concat([base_image, style_reference_image, combination_image], axis = 0)

    #Extracting the features of images
    features = feature_extractor(input_tensor)

    #Initializing the loss variable as scalar (zero tensor)
    loss = tf.zeros(shape = ())

    #Extracting the content features - images with similar content will have the same deep layers
    layer_features = features[content_layer]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]

    #Calculating the content loss
    loss = loss + content_weight * content_loss(base_image_features, combination_features)
    
    #Extracting the style features and adding style loss to overall loss
    for layer_name in style_layers:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_reference_features, combination_features)
        loss = loss + (style_weight / len(style_layers)) * sl

    return loss

Calculating the gradient

In [27]:
#Converting the below function to a Tensorflow graph function - Improved performance
#Function to compute gradients of loss with respect to combination_image
@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = loss_function(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

Pre-processing the image

In [28]:
#Creating a function that uses existing preprocessing techniques from vgg19
#Loading, Resizing and Formatting an image into appropriate tesnor
def preprocess_image(image_path):

  #Loading the image and resizing it
  img = tf.keras.utils.load_img(image_path, target_size = (img_nrows, img_ncols))

  #Converting image to array
  img = tf.keras.utils.img_to_array(img)

  #Matching the batch dimension
  img = np.expand_dims(img, axis = 0)

  img = vgg19.preprocess_input(img)

  return tf.convert_to_tensor(img)

De-processing the image

In [29]:
#Defining a function to convert image tensor to original image format
def deprocess_image(x):

    #Converting the tensor to an array
    x = x.reshape((img_nrows, img_ncols, 3))

    #Adding mean values that were subtracted during pre-processing - default for imagenet trained VGG models
    x[:, :, 0] = x[:, :, 0] + 103.939
    x[:, :, 1] = x[:, :, 1] + 116.779
    x[:, :, 2] = x[:, :, 2] + 123.68

    #Converting from BGR to RGB
    x = x[:, :, ::-1]

    #Ensuring range of pixel values
    x = np.clip(x, 0, 255).astype("uint8")

    return x

Saving the generated image

In [30]:
#Defining a function to save the resulting image 
def result_saver(iteration):
  
  image_name = "generated_image_iteration_" + str(iteration) + '.png'

  #Saving the resulting image
  img = deprocess_image(combination_image.numpy())
  tf.keras.utils.save_img(image_name, img)

Training the model

In [ ]:
#Defining content and style image paths
content_image_path = '/content/content.jpg'
style_image_path = '/content/style.jpg'

#Defining the dimensions of output image
width, height = tf.keras.utils.load_img(content_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

#Defining the SGD optimizer with learning rate and decay rate
optimizer = SGD(tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = 100.0, decay_steps = 100, decay_rate = 0.96))

#Preprocessing the content and style images
base_image = preprocess_image(content_image_path)
style_reference_image = preprocess_image(style_image_path)

#Initializing the resulting image and preprocessing it
combination_image = tf.Variable(preprocess_image(content_image_path))

#Defining the number of iterations for optimization
iterations = 10000

#Defining the optimization loop
for i in range(1, iterations + 1):

  #Computing loss and gradients
  loss, grads = compute_loss_and_grads(combination_image, base_image, style_reference_image)

  #Applying gradients to the result using optimizer
  optimizer.apply_gradients([(grads, combination_image)])
  
  #Prining the Iteration, Loss and Saving the resulting image - for every 5000 transactions
  if i % 1000 == 0:
      print("Iteration %d: loss = %.2f" % (i, loss))
      result_saver(i)